## MNIST and Softmax

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(),
                         download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = batch_size,
                                         shuffle = True,
                                         drop_last = True)

In [6]:
## nn layers
linear1 = torch.nn.Linear(784, 512, bias = True)
linear2 = torch.nn.Linear(512, 512, bias = True)
linear3 = torch.nn.Linear(512, 512, bias = True)
linear4 = torch.nn.Linear(512, 512, bias = True)
linear5 = torch.nn.Linear(512, 10, bias = True)
relu = torch.nn.ReLU()

In [7]:
# xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0565,  0.0423, -0.0155,  ...,  0.1012,  0.0459, -0.0191],
        [ 0.0772,  0.0452, -0.0638,  ...,  0.0476, -0.0638,  0.0528],
        [ 0.0311, -0.1023, -0.0701,  ...,  0.0412, -0.1004,  0.0738],
        ...,
        [ 0.0334,  0.0187, -0.1021,  ...,  0.0280, -0.0583, -0.1018],
        [-0.0506, -0.0939, -0.0467,  ..., -0.0554, -0.0325,  0.0640],
        [-0.0183, -0.0123,  0.1025,  ..., -0.0214,  0.0220, -0.0741]],
       requires_grad=True)

In [8]:
# model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3, relu, linear4, relu, linear5).to(device)

In [9]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [10]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, y in data_loader:
        X = X.view(-1, 28 * 28).to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch : ', '%04d' % (epoch + 1), 'Cost = ', '{:.9f}'.format(avg_cost))
    
print('Learning Finished.')

Epoch :  0001 Cost =  0.209781021
Epoch :  0002 Cost =  0.091920763
Epoch :  0003 Cost =  0.064412147
Epoch :  0004 Cost =  0.050555605
Epoch :  0005 Cost =  0.041137647
Epoch :  0006 Cost =  0.035623103
Epoch :  0007 Cost =  0.029034594
Epoch :  0008 Cost =  0.028723054
Epoch :  0009 Cost =  0.021132756
Epoch :  0010 Cost =  0.023541553
Epoch :  0011 Cost =  0.020767922
Epoch :  0012 Cost =  0.018654220
Epoch :  0013 Cost =  0.018412400
Epoch :  0014 Cost =  0.019621216
Epoch :  0015 Cost =  0.015028213
Learning Finished.


In [12]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    pred = model(X_test)
    correct_pred = torch.argmax(pred, 1) == y_test
    accuracy = correct_pred.float().mean()
    print('Accuracy : ', accuracy.item())
    
    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    y_single_data = mnist_test.test_labels[r:r + 1].to(device)
    
    print('Label : ', y_single_data.item())
    single_pred = model(X_single_data)
    print('Prediction : ', torch.argmax(single_pred, 1).item())

Accuracy :  0.9810000061988831
Label :  5
Prediction :  5
